### Some tests for an Ising model implementation and visualization

The Ising model Hamiltonian is $\mathcal{H} = -J\sum_{\langle i,j \rangle} \sigma_i \sigma_j - \sum_j h\sigma_j$. The partition function is $Z=\sum_{\{\sigma_i\}} e^{-\beta U}$ with $U=-\epsilon\sum_{\langle i,j \rangle} \sigma_i \sigma_j$ (8.38 and 8.39). The partition function sums over each possible lattice states but there are $2^N$ possible lattice states so for a $10\times 10$ lattice, we have $2^{10^2}\approx 10^{30}$ configurations. This means writing a program that attempts to analyze each possible lattice state will not succeed.

The Metropolis algorithim is a method of importance sampling that generates sample satets for a Monte Carlo simulation (integration) with the correct Boltzmann probabilities. This is a natural choice because we should *probably* sample the most likely states the most --- we don't need to sample extremely rare states. (Think of choosing a more clever volume to integrate over instead of a rectangle to approximate the volume of some other shape). **Schroeder uses periodic (torus) BC**.

Why is Metropolis done 100 times per site? Here I implement the pseudo code in Schroeder.

In [1]:
import pygame
from pygame.locals import *
import numpy as np

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
pygame.init()

screen_width=700
screen_height=400
screen=pygame.display.set_mode((screen_width, screen_height))

surface = pygame.Surface((100,100))
rectangle = pygame.Rect(50,50,10,20)
pygame.draw.rect(surface, "red", rectangle)
while True:
    pygame.display.flip()

: 

Im just trying to get pygame to work quickly. Then Ising implementation should be good

In [3]:
import pygame
from pygame.locals import *

pygame.init()
width, height = 640, 480
screen=pygame.display.set_mode((width, height))

player = pygame.images.load("C:\Noah")

while 1:
    # 5 - clear the screen before drawing it again
    screen.fill(0)
    # 6 - draw the screen elements
    screen.blit(player, (100,100))
    # 7 - update the screen
    pygame.display.flip()
    # 8 - loop through the events
    for event in pygame.event.get():
        # check if the event is the X button
        if event.type==pygame.QUIT:
            # if it is quit the game
            pygame.quit()
            exit(0)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: malformed \N character escape (1315872679.py, line 8)